In [48]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [49]:
# Import stock_prices files
df_stock_prices_1 = pd.read_csv('jpx-tokyo-stock-exchange-prediction/train_files/stock_prices_1.csv')
df_stock_prices_2 = pd.read_csv('jpx-tokyo-stock-exchange-prediction/train_files/stock_prices_2.csv')
df_stock_prices_3 = pd.read_csv('jpx-tokyo-stock-exchange-prediction/train_files/stock_prices_3.csv')
df_stock_prices_4 = pd.read_csv('jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')
df_stock_prices = pd.concat([df_stock_prices_1, df_stock_prices_2, df_stock_prices_3, df_stock_prices_4]).reset_index(drop=True)
df_stock_prices

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2602407,20220624_9990,2022-06-24,9990,576.0,576.0,563.0,564.0,24200,1.0,NaN,False,0.027073
2602408,20220624_9991,2022-06-24,9991,810.0,815.0,804.0,815.0,8700,1.0,NaN,False,0.001220
2602409,20220624_9993,2022-06-24,9993,1548.0,1548.0,1497.0,1497.0,12600,1.0,NaN,False,0.001329
2602410,20220624_9994,2022-06-24,9994,2507.0,2527.0,2498.0,2527.0,7300,1.0,NaN,False,0.003185


AdjustmentFactor is the adjustment factor applied to the stock prices and volumes due to stock splits or reverse stock splits.

Take the stock with SecuritiesCode 2987 as an example.
On 2021-12-07, the close price is 3120, volume is 107600, and the AdjustmentFactor column shows 0.5. This indicates a stock split occurred after the close of trading on that day. To adjust the previous day's close price to reflect the stock split and compare it on the same basis as the price on 2021-12-08, you multiply the previous close price by the adjustment factor:

$$
\text{Adjusted Close Price on 2021-12-07} = 3120 \times 0.5 = 1560
$$

$$
\text{Adjusted Volume on 2021-12-07} = 107600 \div 0.5 = 215200
$$

On 2021-12-08, the close price is 1445 and the volume is 253300. These post-split figures can be directly compared to the adjusted figures from the previous day to analyze the stock's performance.


In [50]:
df_stock_prices[(df_stock_prices['Date'].isin(['2021-12-07', '2021-12-08'])) & (df_stock_prices['SecuritiesCode'] == 2987)]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
2334831,20211207_2987,2021-12-07,2987,3465.0,3465.0,3070.0,3120.0,107600,0.5,NaN,False,-0.022837
2336831,20211208_2987,2021-12-08,2987,1570.0,1579.0,1433.0,1445.0,253300,1.0,NaN,False,-0.030453


In [51]:
# List of securities, used later in the loop
securities_list = df_stock_prices['SecuritiesCode'].unique()
securities_list

array([1301, 1332, 1333, ..., 4168, 7342, 4169], dtype=int64)

In [52]:
# Check for blank values in Volume - passed
print(df_stock_prices.isna().sum())

# Check for data types - passed
print(df_stock_prices.dtypes)

# Check for any non-positive values in Volume - found zero values
print(df_stock_prices[df_stock_prices['Volume'] == 0])
print(df_stock_prices[df_stock_prices['Volume'] < 0])


# Remove zero values in Volume
df_stock_prices = df_stock_prices[df_stock_prices['Volume'] > 0].reset_index(drop=True)

df_stock_prices

RowId                     0
Date                      0
SecuritiesCode            0
Open                   8426
High                   8426
Low                    8426
Close                  8426
Volume                    0
AdjustmentFactor          0
ExpectedDividend    2581536
SupervisionFlag           0
Target                  246
dtype: int64
RowId                object
Date                 object
SecuritiesCode        int64
Open                float64
High                float64
Low                 float64
Close               float64
Volume                int64
AdjustmentFactor    float64
ExpectedDividend    float64
SupervisionFlag        bool
Target              float64
dtype: object
                 RowId        Date  SecuritiesCode  Open  High  Low  Close  \
401      20170104_3540  2017-01-04            3540   NaN   NaN  NaN    NaN   
1753     20170104_9539  2017-01-04            9539   NaN   NaN  NaN    NaN   
2266     20170105_3540  2017-01-05            3540   NaN   NaN  NaN

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2593981,20220624_9990,2022-06-24,9990,576.0,576.0,563.0,564.0,24200,1.0,NaN,False,0.027073
2593982,20220624_9991,2022-06-24,9991,810.0,815.0,804.0,815.0,8700,1.0,NaN,False,0.001220
2593983,20220624_9993,2022-06-24,9993,1548.0,1548.0,1497.0,1497.0,12600,1.0,NaN,False,0.001329
2593984,20220624_9994,2022-06-24,9994,2507.0,2527.0,2498.0,2527.0,7300,1.0,NaN,False,0.003185


In [53]:
df_stock_prices_adjusted = df_stock_prices.copy()
df_stock_prices_adjusted['AdjustmentFactor'] = (
    df_stock_prices_adjusted.sort_values(['SecuritiesCode', 'Date'], ascending=[True, False])
    .groupby('SecuritiesCode')['AdjustmentFactor']
    .cumprod()
)

df_stock_prices_adjusted = df_stock_prices_adjusted.sort_values(['SecuritiesCode', 'Date'])

In [54]:
# Test the function on the stock with SecuritiesCode 3475
print(df_stock_prices[(df_stock_prices['SecuritiesCode'] == 3475) & (df_stock_prices['AdjustmentFactor'] != 1)])

df_stock_prices_adjusted[df_stock_prices_adjusted['SecuritiesCode'] == 3475]
# Shows correct AdjustmentFactor column

                 RowId        Date  SecuritiesCode    Open    High     Low  \
143828   20170425_3475  2017-04-25            3475  2149.0  2185.0  2138.0   
338618   20170926_3475  2017-09-26            3475  1761.0  1810.0  1761.0   
1786920  20201028_3475  2020-10-28            3475  3075.0  3225.0  3070.0   

          Close  Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag  \
143828   2180.0    8100               0.5               NaN            False   
338618   1805.0   36200               0.5               NaN            False   
1786920  3075.0  221400               0.5               NaN            False   

           Target  
143828   0.028174  
338618   0.024218  
1786920 -0.106927  


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
399,20170104_3475,2017-01-04,3475,2063.0,2145.0,2055.0,2100.0,31000,0.125,NaN,False,-0.018560
2262,20170105_3475,2017-01-05,3475,2099.0,2198.0,2085.0,2155.0,38300,0.125,NaN,False,-0.005295
4125,20170106_3475,2017-01-06,3475,2154.0,2154.0,2061.0,2115.0,24500,0.125,NaN,False,-0.014829
5988,20170110_3475,2017-01-10,3475,2099.0,2115.0,2083.0,2104.0,8900,0.125,NaN,False,-0.011964
7848,20170111_3475,2017-01-11,3475,2109.0,2109.0,2050.0,2073.0,22500,0.125,NaN,False,-0.011328
...,...,...,...,...,...,...,...,...,...,...,...,...
2584434,20220620_3475,2022-06-20,3475,1251.0,1251.0,1153.0,1163.0,200800,1.000,NaN,False,0.067416
2586424,20220621_3475,2022-06-21,3475,1160.0,1192.0,1120.0,1157.0,184900,1.000,NaN,False,0.015385
2588415,20220622_3475,2022-06-22,3475,1187.0,1254.0,1181.0,1235.0,186800,1.000,NaN,False,0.011962
2590407,20220623_3475,2022-06-23,3475,1235.0,1279.0,1235.0,1254.0,150600,1.000,NaN,False,0.000000


In [55]:
# Calculate adjusted volume and adjusted close
df_stock_prices_adjusted['AdjustedVolume'] = df_stock_prices_adjusted['Volume'] / df_stock_prices_adjusted['AdjustmentFactor']
df_stock_prices_adjusted['AdjustedClose'] = df_stock_prices_adjusted['Close'] * df_stock_prices_adjusted['AdjustmentFactor']

df_stock_prices_adjusted[df_stock_prices_adjusted['SecuritiesCode'] == 3475]
# Show correct numbers vs direct calculations in Excel

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,AdjustedVolume,AdjustedClose
399,20170104_3475,2017-01-04,3475,2063.0,2145.0,2055.0,2100.0,31000,0.125,NaN,False,-0.018560,248000.0,262.500
2262,20170105_3475,2017-01-05,3475,2099.0,2198.0,2085.0,2155.0,38300,0.125,NaN,False,-0.005295,306400.0,269.375
4125,20170106_3475,2017-01-06,3475,2154.0,2154.0,2061.0,2115.0,24500,0.125,NaN,False,-0.014829,196000.0,264.375
5988,20170110_3475,2017-01-10,3475,2099.0,2115.0,2083.0,2104.0,8900,0.125,NaN,False,-0.011964,71200.0,263.000
7848,20170111_3475,2017-01-11,3475,2109.0,2109.0,2050.0,2073.0,22500,0.125,NaN,False,-0.011328,180000.0,259.125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584434,20220620_3475,2022-06-20,3475,1251.0,1251.0,1153.0,1163.0,200800,1.000,NaN,False,0.067416,200800.0,1163.000
2586424,20220621_3475,2022-06-21,3475,1160.0,1192.0,1120.0,1157.0,184900,1.000,NaN,False,0.015385,184900.0,1157.000
2588415,20220622_3475,2022-06-22,3475,1187.0,1254.0,1181.0,1235.0,186800,1.000,NaN,False,0.011962,186800.0,1235.000
2590407,20220623_3475,2022-06-23,3475,1235.0,1279.0,1235.0,1254.0,150600,1.000,NaN,False,0.000000,150600.0,1254.000


In [56]:
# Calculate monthly volume
df_stock_prices_adjusted['Date'] = pd.to_datetime(df_stock_prices_adjusted['Date'])
df_stock_prices_adjusted['Date'] = df_stock_prices_adjusted['Date'].dt.to_period('M')
df_stock_monthly_volume = df_stock_prices_adjusted.copy()
df_stock_monthly_volume = df_stock_prices_adjusted.groupby(['Date', 'SecuritiesCode'])['AdjustedVolume'].sum().reset_index()
df_stock_monthly_volume

,Date,SecuritiesCode,AdjustedVolume
0,2017-01,1301,357500.0
1,2017-01,1332,48829700.0
2,2017-01,1333,4545400.0
3,2017-01,1376,132200.0
4,2017-01,1377,4264300.0
...,...,...,...
128546,2022-06,9990,912300.0
128547,2022-06,9991,370500.0
128548,2022-06,9993,77800.0
128549,2022-06,9994,96100.0


In [57]:
# Calculate monthly return
df_stock_monthly_return = df_stock_prices_adjusted.copy()
df_stock_monthly_return = df_stock_monthly_return.groupby(['Date', 'SecuritiesCode'])['AdjustedClose'].last().reset_index()
df_stock_monthly_return['AdjustedReturn'] = (df_stock_monthly_return.groupby('SecuritiesCode')['AdjustedClose'].pct_change())
df_stock_monthly_return = df_stock_monthly_return[df_stock_monthly_return['Date'] != df_stock_monthly_return['Date'].min()].reset_index(drop=True)
df_stock_monthly_return

,Date,SecuritiesCode,AdjustedClose,AdjustedReturn
0,2017-02,1301,2971.0,0.091477
1,2017-02,1332,568.0,0.021583
2,2017-02,1333,3365.0,0.078526
3,2017-02,1376,1450.0,-0.020932
4,2017-02,1377,3165.0,-0.021638
...,...,...,...,...
126682,2022-06,9990,564.0,-0.050505
126683,2022-06,9991,815.0,-0.016888
126684,2022-06,9993,1497.0,-0.013184
126685,2022-06,9994,2527.0,0.020186


In [58]:
# Calculate monthly log_volume for individual stocks and for the market
df_stock_log_volume = df_stock_monthly_volume.copy()
df_stock_log_volume['log_volume'] = np.log(df_stock_log_volume['AdjustedVolume'])
df_stock_log_volume['market_volume'] = df_stock_log_volume.groupby('Date')['AdjustedVolume'].transform('sum')
df_stock_log_volume['market_log_volume'] = np.log(df_stock_log_volume['market_volume'])
df_stock_log_volume

,Date,SecuritiesCode,AdjustedVolume,log_volume,market_volume,market_log_volume
0,2017-01,1301,357500.0,12.786891,2.327261e+10,23.870543
1,2017-01,1332,48829700.0,17.703849,2.327261e+10,23.870543
2,2017-01,1333,4545400.0,15.329626,2.327261e+10,23.870543
3,2017-01,1376,132200.0,11.792071,2.327261e+10,23.870543
4,2017-01,1377,4264300.0,15.265789,2.327261e+10,23.870543
...,...,...,...,...,...,...
128546,2022-06,9990,912300.0,13.723724,2.319006e+10,23.866990
128547,2022-06,9991,370500.0,12.822609,2.319006e+10,23.866990
128548,2022-06,9993,77800.0,11.261897,2.319006e+10,23.866990
128549,2022-06,9994,96100.0,11.473145,2.319006e+10,23.866990


In [59]:
# Only take the necessary columns for regression
y = df_stock_log_volume[['SecuritiesCode', 'Date', 'log_volume']]
y

,SecuritiesCode,Date,log_volume
0,1301,2017-01,12.786891
1,1332,2017-01,17.703849
2,1333,2017-01,15.329626
3,1376,2017-01,11.792071
4,1377,2017-01,15.265789
...,...,...,...
128546,9990,2022-06,13.723724
128547,9991,2022-06,12.822609
128548,9993,2022-06,11.261897
128549,9994,2022-06,11.473145


In [60]:
# Take the market_log_volume time series for regression
X = df_stock_log_volume[['Date', 'market_log_volume']].drop_duplicates()
X = X.set_index('Date')
X = X[['market_log_volume']]
X = sm.add_constant(X)
X

,const,market_log_volume
Date,,
2017-01,1.0,23.870543
2017-02,1.0,23.931984
2017-03,1.0,23.989407
2017-04,1.0,23.926133
2017-05,1.0,23.959802
...,...,...
2022-02,1.0,23.906400
2022-03,1.0,24.182678
2022-04,1.0,23.883846


An Ordinary Least Squares (OLS) regression model is used.

$$
V_{i,t} = A_i + B_i V_{m,t} + \varepsilon_{i,t}
$$

where:
- $V_{i,t}$ is the log of volume for stock $i$ in month $t$,
- $V_{m,t}$ is the log of total market volume for month $t$,
- $A_i$ is the intercept,
- $B_i$ is the slope coefficient, quantifying the sensitivity of the stock's log-volume to changes in the market log-volume,
- $\varepsilon_{i,t}$ is the residual, interpreted as the abnormal trading volume (ATV) of stock $i$ in month $t$.

The objective function to minimise is:
$$
S = \sum_{t=1}^T \varepsilon_{i,t}^2 = \sum_{t=1}^T (V_{i,t} - A_i - B_i V_{m,t})^2
$$

Taking partial derivatives with respect to $A_i$:
$$
\frac{\partial S}{\partial A_i} = -2 \sum_{t=1}^T (V_{i,t} - A_i - B_i V_{m,t}) = 0
$$

$$
A_i = \frac{1}{T}\sum_{t=1}^T V_{i,t} - B_i \frac{1}{T}\sum_{t=1}^T V_{m,t}
$$

$$
A_i = \bar{V}_{i,t} - B_i \bar{V}_{m,t}
$$

Taking partial derivatives with respect to $B_i$:
$$
\frac{\partial S}{\partial B_i} = -2 \sum_{t=1}^T (V_{i,t} - A_i - B_i V_{m,t}) V_{m,t} = 0
$$

Subsititute $A_i = \bar{V}_{i,t} - B_i \bar{V}_{m,t}$, we get:

$$
B_i = \frac{\sum_{t=1}^T (V_{i,t} - \bar{V}_{i,t})(V_{m,t} - \bar{V}_{m,t})}{\sum_{t=1}^T (V_{m,t} - \bar{V}_{m,t})^2}
$$

The residual is given by:
$$
\varepsilon_{i,t} = V_{i,t} - (A_i + B_i V_{m,t})
$$

We loop over all stocks to obtain a time series of ATVs for each stock.

In [61]:
# Run the time series regression on each stock and produce the residuals
def compute_residuals(group):
    y_i = group['log_volume']
    X_subset = X.loc[group.index]
    model = sm.OLS(y_i, X_subset)
    results = model.fit()
    return results.resid


residuals = (y.set_index('Date').groupby('SecuritiesCode').apply(compute_residuals))

df_atv = residuals.unstack(level=0)
df_atv

C:\Users\Angus Cheung\AppData\Local\Temp\ipykernel_31420\2841675932.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  residuals = (y.set_index('Date').groupby('SecuritiesCode').apply(compute_residuals))


SecuritiesCode,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9982,9983,9984,9987,9989,9990,9991,9993,9994,9997
Date,,,,,,,,,,,,,,,,,,,,,
2017-01,-0.175812,0.129911,0.097639,NaN,0.033716,0.693322,-0.214240,0.037344,-0.318635,NaN,...,-0.529169,0.189782,0.155106,-0.019926,0.282309,0.525512,0.345386,-0.133552,-0.737431,-0.445626
2017-02,0.254373,0.285503,0.313747,NaN,-0.175386,-0.044255,-0.099336,-0.624168,-0.920741,NaN,...,0.571964,-0.318280,-0.062948,0.043890,0.323350,0.125964,0.279265,0.550815,-0.701010,-0.363318
2017-03,1.312111,0.237043,0.152017,NaN,0.020491,0.233545,0.072736,0.076991,-0.693133,-1.789372,...,-0.284758,-0.095432,-0.301798,0.341563,0.156583,0.106486,0.097513,-0.390367,-1.166112,-0.405048
2017-04,0.348895,0.157044,0.466417,NaN,-0.046740,0.668797,-0.105163,0.429237,-0.308875,-1.102864,...,-0.218353,-0.012215,-0.223042,0.328015,-0.091005,-0.177901,0.400649,-0.432367,-1.093852,0.411574
2017-05,0.454951,0.144402,0.393423,NaN,-0.304467,0.767087,-0.142027,0.397202,-0.723519,-1.385833,...,-0.375421,-0.297817,-0.095969,0.428911,-0.231700,0.269736,0.345142,-0.479556,-0.976878,0.524619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02,-0.501177,0.025409,-0.169800,-0.513531,-0.042791,-0.241433,-0.362669,-1.175043,0.590363,0.920480,...,1.119852,-0.445713,0.076215,-0.516062,-0.228714,0.146713,-0.545857,0.723397,-0.474332,-0.203980
2022-03,0.335113,-0.216807,-0.046041,-0.420968,0.378099,-0.089879,0.020188,-0.808849,0.710361,1.068343,...,0.828460,-0.387816,0.115579,-0.186742,-0.122949,0.719604,-0.417918,-0.139011,0.063523,0.133771
2022-04,-0.178998,-0.446894,-0.276917,-0.470003,0.879520,0.636023,-0.036580,-0.682537,0.561568,0.424058,...,0.327947,-0.004405,0.076790,-0.441168,0.242397,-0.117940,-0.271783,-0.161971,-0.646031,-0.476767


In [62]:
df_atv.loc['2022-03']

SecuritiesCode
1301    0.335113
1332   -0.216807
1333   -0.046041
1375   -0.420968
1376    0.378099
          ...   
9990    0.719604
9991   -0.417918
9993   -0.139011
9994    0.063523
9997    0.133771
Name: 2022-03, Length: 2000, dtype: float64

In [63]:
# Assigning quintiles to ATVs
df_atv_group = df_atv.copy()

df_atv_group[:] = pd.DataFrame(
    np.vstack([pd.qcut(row, 5, labels=False, duplicates='drop') for row in df_atv_group.values]),index=df_atv_group.index,columns=df_atv_group.columns)

df_atv_group.index = (df_atv_group.index.to_timestamp() + pd.DateOffset(months=1)).to_period('M')

df_atv_group

SecuritiesCode,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9982,9983,9984,9987,9989,9990,9991,9993,9994,9997
Date,,,,,,,,,,,,,,,,,,,,,
2017-02,1.0,3.0,3.0,NaN,2.0,4.0,1.0,2.0,1.0,NaN,...,0.0,3.0,3.0,2.0,3.0,4.0,4.0,1.0,0.0,0.0
2017-03,3.0,3.0,3.0,NaN,1.0,2.0,2.0,0.0,0.0,NaN,...,4.0,1.0,2.0,2.0,3.0,2.0,3.0,4.0,0.0,1.0
2017-04,4.0,3.0,3.0,NaN,2.0,3.0,2.0,2.0,0.0,0.0,...,1.0,1.0,1.0,3.0,3.0,2.0,2.0,0.0,0.0,0.0
2017-05,3.0,3.0,4.0,NaN,2.0,4.0,2.0,4.0,1.0,0.0,...,1.0,2.0,1.0,3.0,2.0,1.0,4.0,0.0,0.0,4.0
2017-06,3.0,2.0,3.0,NaN,1.0,4.0,1.0,3.0,0.0,0.0,...,0.0,1.0,1.0,3.0,1.0,3.0,3.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,1.0,3.0,2.0,1.0,2.0,2.0,1.0,0.0,4.0,4.0,...,4.0,1.0,3.0,1.0,2.0,3.0,1.0,4.0,1.0,2.0
2022-04,4.0,1.0,2.0,1.0,4.0,2.0,3.0,0.0,4.0,4.0,...,4.0,1.0,3.0,2.0,2.0,4.0,1.0,2.0,3.0,3.0
2022-05,2.0,1.0,1.0,1.0,4.0,4.0,2.0,0.0,4.0,4.0,...,4.0,2.0,3.0,1.0,3.0,2.0,1.0,2.0,0.0,1.0


In [64]:
# Add labels O and U
df_atv_label = df_atv_group.copy().astype(object)

df_atv_label[:] = np.where(df_atv_label == 4, 'O', np.where(df_atv_label == 0, 'U', df_atv_label))

df_atv_label

SecuritiesCode,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9982,9983,9984,9987,9989,9990,9991,9993,9994,9997
Date,,,,,,,,,,,,,,,,,,,,,
2017-02,1.0,3.0,3.0,NaN,2.0,O,1.0,2.0,1.0,NaN,...,U,3.0,3.0,2.0,3.0,O,O,1.0,U,U
2017-03,3.0,3.0,3.0,NaN,1.0,2.0,2.0,U,U,NaN,...,O,1.0,2.0,2.0,3.0,2.0,3.0,O,U,1.0
2017-04,O,3.0,3.0,NaN,2.0,3.0,2.0,2.0,U,U,...,1.0,1.0,1.0,3.0,3.0,2.0,2.0,U,U,U
2017-05,3.0,3.0,O,NaN,2.0,O,2.0,O,1.0,U,...,1.0,2.0,1.0,3.0,2.0,1.0,O,U,U,O
2017-06,3.0,2.0,3.0,NaN,1.0,O,1.0,3.0,U,U,...,U,1.0,1.0,3.0,1.0,3.0,3.0,U,U,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,1.0,3.0,2.0,1.0,2.0,2.0,1.0,U,O,O,...,O,1.0,3.0,1.0,2.0,3.0,1.0,O,1.0,2.0
2022-04,O,1.0,2.0,1.0,O,2.0,3.0,U,O,O,...,O,1.0,3.0,2.0,2.0,O,1.0,2.0,3.0,3.0
2022-05,2.0,1.0,1.0,1.0,O,O,2.0,U,O,O,...,O,2.0,3.0,1.0,3.0,2.0,1.0,2.0,U,1.0


In [65]:
# One hot encoding of O and U
df_U_one_hot = df_atv_label.where(df_atv_label == 'U', np.nan)
df_U_one_hot = pd.get_dummies(df_U_one_hot, columns=df_U_one_hot.columns)
df_O_one_hot = df_atv_label.where(df_atv_label == 'O', np.nan)
df_O_one_hot = pd.get_dummies(df_O_one_hot, columns=df_O_one_hot.columns)
df_U_one_hot

,1301_U,1332_U,1333_U,1375_U,1376_U,1377_U,1379_U,1381_U,1407_U,1413_U,...,9982_U,9983_U,9984_U,9987_U,9989_U,9990_U,9991_U,9993_U,9994_U,9997_U
Date,,,,,,,,,,,,,,,,,,,,,
2017-02,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
2017-03,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,True,False
2017-04,False,False,False,False,False,False,False,False,True,True,...,False,False,False,False,False,False,False,True,True,True
2017-05,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,True,False
2017-06,False,False,False,False,False,False,False,False,True,True,...,True,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2022-04,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2022-05,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [66]:
# Get market cap data
df_stock_list = pd.read_csv('jpx-tokyo-stock-exchange-prediction/train_files/stock_list.csv')
df_stock_list = df_stock_list[['SecuritiesCode', 'EffectiveDate', 'MarketCapitalization']]
df_stock_list['Date'] = pd.to_datetime(df_stock_list['EffectiveDate'], format='%Y%m%d')
df_stock_list['Date'] = df_stock_list['Date'].dt.to_period('M')
df_stock_list

,SecuritiesCode,EffectiveDate,MarketCapitalization,Date
0,1301,20211230,3.365911e+10,2021-12
1,1305,20211230,7.621831e+12,2021-12
2,1306,20211230,1.641739e+13,2021-12
3,1308,20211230,7.671945e+12,2021-12
4,1309,20211230,3.216145e+09,2021-12
...,...,...,...,...
4412,9994,20211230,2.654474e+10,2021-12
4413,9995,20211230,1.083499e+10,2021-12
4414,9996,20211230,1.361913e+10,2021-12
4415,9997,20211230,6.894633e+10,2021-12


In [67]:
# Prepare data frame for market cap approximation
df_market_cap = df_stock_monthly_return.copy()

df_market_cap = pd.merge(df_market_cap, df_stock_list, on=['SecuritiesCode', 'Date'], how='left')
df_market_cap

,Date,SecuritiesCode,AdjustedClose,AdjustedReturn,EffectiveDate,MarketCapitalization
0,2017-02,1301,2971.0,0.091477,NaN,NaN
1,2017-02,1332,568.0,0.021583,NaN,NaN
2,2017-02,1333,3365.0,0.078526,NaN,NaN
3,2017-02,1376,1450.0,-0.020932,NaN,NaN
4,2017-02,1377,3165.0,-0.021638,NaN,NaN
...,...,...,...,...,...,...
126682,2022-06,9990,564.0,-0.050505,NaN,NaN
126683,2022-06,9991,815.0,-0.016888,NaN,NaN
126684,2022-06,9993,1497.0,-0.013184,NaN,NaN
126685,2022-06,9994,2527.0,0.020186,NaN,NaN


In [68]:
# Market cap approximation: Market cap only available for 2021-12. Use adjusted returns as a proxy for change in market cap
def fill_forward(group):
    group = group.sort_values('Date')

    mask = group['MarketCapitalization'].isna() & (group['Date'] > '2021-12')
    group['MarketCapitalization'] = group['MarketCapitalization'].ffill()
    group.loc[mask, 'MarketCapitalization'] *= (1 + group.loc[mask, 'AdjustedReturn']).cumprod()

    return group


def fill_backward(group):
    group = group.sort_values('Date', ascending=False)

    mask = group['MarketCapitalization'].isna() & (group['Date'] < '2021-12')
    group['MarketCapitalization'] = group['MarketCapitalization'].ffill()
    group.loc[mask, 'MarketCapitalization'] /= (1 + group.loc[mask, 'AdjustedReturn']).cumprod()

    return group

df_market_cap = df_market_cap.groupby('SecuritiesCode').apply(fill_forward)
df_market_cap = df_market_cap.reset_index(drop=True)
df_market_cap = df_market_cap.groupby('SecuritiesCode').apply(fill_backward)
df_market_cap = df_market_cap.reset_index(drop=True)
df_market_cap = df_market_cap.sort_values(['Date', 'SecuritiesCode']).set_index('Date', drop=True)
df_market_cap

C:\Users\Angus Cheung\AppData\Local\Temp\ipykernel_31420\217296310.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_market_cap = df_market_cap.groupby('SecuritiesCode').apply(fill_forward)
C:\Users\Angus Cheung\AppData\Local\Temp\ipykernel_31420\217296310.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_market_cap = df_market_cap.groupby('SecuritiesCode').apply(fill_backward)


,SecuritiesCode,AdjustedClose,AdjustedReturn,EffectiveDate,MarketCapitalization
Date,,,,,
2017-02,1301,2971.0,0.091477,NaN,3.159314e+10
2017-02,1332,568.0,0.021583,NaN,1.646164e+11
2017-02,1333,3365.0,0.078526,NaN,1.735250e+11
2017-02,1376,1450.0,-0.020932,NaN,1.945299e+10
2017-02,1377,3165.0,-0.021638,NaN,1.612265e+11
...,...,...,...,...,...
2022-06,9990,564.0,-0.050505,NaN,1.684098e+10
2022-06,9991,815.0,-0.016888,NaN,2.969544e+10
2022-06,9993,1497.0,-0.013184,NaN,1.640836e+10


In [69]:
# Check calculations
df_market_cap[df_market_cap['SecuritiesCode'] == 3475]
# Show correct numbers vs direct calculations in Exccel

,SecuritiesCode,AdjustedClose,AdjustedReturn,EffectiveDate,MarketCapitalization
Date,,,,,
2017-02,3475,280.125,0.092105,NaN,3.715660e+09
2017-03,3475,297.375,0.061580,NaN,4.057892e+09
2017-04,3475,281.000,-0.055065,NaN,4.307776e+09
2017-05,3475,264.750,-0.057829,NaN,4.070567e+09
2017-06,3475,273.750,0.033994,NaN,3.835170e+09
...,...,...,...,...,...
2022-02,3475,1156.000,0.082397,NaN,1.760496e+10
2022-03,3475,1047.000,-0.094291,NaN,1.594497e+10
2022-04,3475,1059.000,0.011461,NaN,1.612772e+10


In [70]:
# Assigning quintiles to market cap
df_market_cap_values = df_market_cap.pivot(columns='SecuritiesCode', values='MarketCapitalization')
df_market_cap_group = df_market_cap_values.copy()

df_market_cap_group[:] = pd.DataFrame(
    np.vstack([pd.qcut(row, 5, labels=False, duplicates='drop') for row in df_market_cap_group.values]),index=df_market_cap_group.index,columns=df_market_cap_group.columns)

df_market_cap_group.index = (df_market_cap_group.index.to_timestamp() + pd.DateOffset(months=1)).to_period('M')

df_market_cap_group

SecuritiesCode,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9982,9983,9984,9987,9989,9990,9991,9993,9994,9997
Date,,,,,,,,,,,,,,,,,,,,,
2017-03,1.0,3.0,3.0,NaN,0.0,3.0,2.0,0.0,0.0,NaN,...,0.0,4.0,4.0,4.0,4.0,1.0,2.0,0.0,0.0,2.0
2017-04,1.0,3.0,3.0,NaN,0.0,3.0,2.0,0.0,0.0,NaN,...,0.0,4.0,4.0,4.0,4.0,1.0,2.0,0.0,0.0,2.0
2017-05,1.0,3.0,3.0,NaN,0.0,3.0,2.0,0.0,0.0,1.0,...,0.0,4.0,4.0,4.0,4.0,1.0,1.0,0.0,0.0,2.0
2017-06,1.0,3.0,3.0,NaN,0.0,3.0,2.0,0.0,0.0,1.0,...,0.0,4.0,4.0,4.0,4.0,1.0,2.0,0.0,0.0,3.0
2017-07,1.0,3.0,3.0,NaN,0.0,3.0,2.0,0.0,0.0,1.0,...,0.0,4.0,4.0,4.0,4.0,1.0,1.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,1.0,3.0,3.0,2.0,0.0,3.0,2.0,0.0,3.0,1.0,...,0.0,4.0,4.0,4.0,4.0,0.0,1.0,0.0,1.0,2.0
2022-04,1.0,3.0,3.0,2.0,0.0,3.0,2.0,0.0,3.0,1.0,...,0.0,4.0,4.0,4.0,4.0,0.0,1.0,0.0,1.0,2.0
2022-05,1.0,3.0,3.0,2.0,0.0,3.0,2.0,0.0,3.0,1.0,...,0.0,4.0,4.0,4.0,4.0,0.0,1.0,0.0,1.0,2.0


In [71]:
# Add labels B and S
df_market_cap_label = df_market_cap_group.copy().astype(object)

df_market_cap_label[:] = np.where(df_market_cap_label == 4, 'B', np.where(df_market_cap_label == 0, 'S', df_market_cap_label))

df_market_cap_label

SecuritiesCode,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9982,9983,9984,9987,9989,9990,9991,9993,9994,9997
Date,,,,,,,,,,,,,,,,,,,,,
2017-03,1.0,3.0,3.0,NaN,S,3.0,2.0,S,S,NaN,...,S,B,B,B,B,1.0,2.0,S,S,2.0
2017-04,1.0,3.0,3.0,NaN,S,3.0,2.0,S,S,NaN,...,S,B,B,B,B,1.0,2.0,S,S,2.0
2017-05,1.0,3.0,3.0,NaN,S,3.0,2.0,S,S,1.0,...,S,B,B,B,B,1.0,1.0,S,S,2.0
2017-06,1.0,3.0,3.0,NaN,S,3.0,2.0,S,S,1.0,...,S,B,B,B,B,1.0,2.0,S,S,3.0
2017-07,1.0,3.0,3.0,NaN,S,3.0,2.0,S,S,1.0,...,S,B,B,B,B,1.0,1.0,S,S,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,1.0,3.0,3.0,2.0,S,3.0,2.0,S,3.0,1.0,...,S,B,B,B,B,S,1.0,S,1.0,2.0
2022-04,1.0,3.0,3.0,2.0,S,3.0,2.0,S,3.0,1.0,...,S,B,B,B,B,S,1.0,S,1.0,2.0
2022-05,1.0,3.0,3.0,2.0,S,3.0,2.0,S,3.0,1.0,...,S,B,B,B,B,S,1.0,S,1.0,2.0


In [72]:
df_S_one_hot = df_market_cap_label.where(df_market_cap_label == 'S', np.nan)
df_S_one_hot = pd.get_dummies(df_S_one_hot, columns=df_S_one_hot.columns)
df_B_one_hot = df_market_cap_label.where(df_market_cap_label == 'B', np.nan)
df_B_one_hot = pd.get_dummies(df_B_one_hot, columns=df_B_one_hot.columns)
df_S_one_hot

,1376_S,1381_S,1407_S,1413_S,1419_S,1429_S,1431_S,1435_S,1518_S,1712_S,...,9903_S,9906_S,9932_S,9955_S,9977_S,9979_S,9982_S,9990_S,9993_S,9994_S
Date,,,,,,,,,,,,,,,,,,,,,
2017-03,True,True,True,False,True,True,False,False,False,False,...,True,True,True,True,True,False,True,False,True,True
2017-04,True,True,True,False,True,True,False,False,False,True,...,True,True,True,True,True,False,True,False,True,True
2017-05,True,True,True,False,True,True,False,False,False,True,...,True,True,True,True,True,False,True,False,True,True
2017-06,True,True,True,False,True,True,False,False,False,True,...,True,True,True,True,True,False,True,False,True,True
2017-07,True,True,True,False,True,True,False,False,False,True,...,True,True,True,True,True,False,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,True,True,False,False,False,True,True,True,True,True,...,True,True,False,False,True,True,True,True,True,False
2022-04,True,True,False,False,False,True,True,True,False,True,...,True,True,False,False,True,True,True,True,True,False
2022-05,True,True,False,False,False,True,True,True,False,True,...,True,True,False,False,True,True,True,True,True,False


In [73]:
# Create signals
df_signals = df_market_cap_label.astype(str) + df_atv_label.astype(str)
df_signals = df_signals.dropna()
df_signals

SecuritiesCode,1301,1332,1333,1375,1376,1377,1379,1381,1407,1413,...,9982,9983,9984,9987,9989,9990,9991,9993,9994,9997
Date,,,,,,,,,,,,,,,,,,,,,
2017-03,1.03.0,3.03.0,3.03.0,nannan,S1.0,3.02.0,2.02.0,SU,SU,nannan,...,SO,B1.0,B2.0,B2.0,B3.0,1.02.0,2.03.0,SO,SU,2.01.0
2017-04,1.0O,3.03.0,3.03.0,nannan,S2.0,3.03.0,2.02.0,S2.0,SU,nanU,...,S1.0,B1.0,B1.0,B3.0,B3.0,1.02.0,2.02.0,SU,SU,2.0U
2017-05,1.03.0,3.03.0,3.0O,nannan,S2.0,3.0O,2.02.0,SO,S1.0,1.0U,...,S1.0,B2.0,B1.0,B3.0,B2.0,1.01.0,1.0O,SU,SU,2.0O
2017-06,1.03.0,3.02.0,3.03.0,nannan,S1.0,3.0O,2.01.0,S3.0,SU,1.0U,...,SU,B1.0,B1.0,B3.0,B1.0,1.03.0,2.03.0,SU,SU,3.0O
2017-07,1.03.0,3.03.0,3.02.0,nannan,S1.0,3.03.0,2.03.0,SO,SU,1.0U,...,SU,B1.0,B2.0,B3.0,B1.0,1.03.0,1.0O,S1.0,SU,3.0O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03,1.01.0,3.03.0,3.02.0,2.01.0,S2.0,3.02.0,2.01.0,SU,3.0O,1.0O,...,SO,B1.0,B3.0,B1.0,B2.0,S3.0,1.01.0,SO,1.01.0,2.02.0
2022-04,1.0O,3.01.0,3.02.0,2.01.0,SO,3.02.0,2.03.0,SU,3.0O,1.0O,...,SO,B1.0,B3.0,B2.0,B2.0,SO,1.01.0,S2.0,1.03.0,2.03.0
2022-05,1.02.0,3.01.0,3.01.0,2.01.0,SO,3.0O,2.02.0,SU,3.0O,1.0O,...,SO,B2.0,B3.0,B1.0,B3.0,S2.0,1.01.0,S2.0,1.0U,2.01.0


In [74]:
df_market_cap['Return*MarketCap'] = df_market_cap['AdjustedReturn'] * df_market_cap['MarketCapitalization']

df_signals_long = df_signals.stack().reset_index()
df_signals_long.columns = ['Date', 'SecuritiesCode', 'Signal']

df_merged = pd.merge(df_signals_long, df_market_cap, on=['Date', 'SecuritiesCode'], how='left')

grouped = df_merged.groupby(['Date', 'Signal']).agg(TotalMarketCap=('MarketCapitalization', 'sum'),TotalReturnMarketCap=('Return*MarketCap', 'sum')).reset_index()

grouped['WeightedReturn'] = grouped['TotalReturnMarketCap'] / grouped['TotalMarketCap']

pivoted = grouped.pivot(index='Date', columns='Signal', values='WeightedReturn').reset_index()

pivoted['TradingFactor'] = ((pivoted['SU'] + pivoted['BU']) - (pivoted['SO'] + pivoted['BO'])) / 2

trading_factors = pivoted['TradingFactor'].tolist()

In [75]:
df_trading_factor = pd.DataFrame(trading_factors, index=df_signals.index, columns=['trading_factor']).dropna()
df_trading_factor

,trading_factor
Date,
2017-03,0.014159
2017-04,0.000889
2017-05,-0.005389
2017-06,0.001682
2017-07,0.003910
...,...
2022-02,-0.003533
2022-03,-0.011618
2022-04,0.013693
